In [219]:
import os
import z_function_list
from functools import reduce

In [220]:
base_path,data,outputs = z_function_list.folder_setup()

In [222]:
os.chdir(data)

In [235]:
#pull in excel file
cleanstat = pd.ExcelFile('clean_comparison.xlsx')
#create dictionary of frames for each sheet
d = {n:cleanstat.parse(n) for n in cleanstat.sheet_names}
#merge into single DF
combined_quarters = reduce(lambda x,y: pd.merge(x,y,on='SegmentID'),[d[sheet] for sheet in cleanstat.sheet_names])

#replace columns names to make more specific
combined_quarters.columns = combined_quarters.columns.str.replace('_x','_Q1_2016').str.replace('_y','_Q4_2016')
column_replacement = {n:k for n,k in zip(combined_quarters.columns[-5:],[x+'_Q4_2017' for x  in combined_quarters.columns[-5:]])}
combined_quarters = combined_quarters.rename(columns=column_replacement)

#add in missing CD

missing_cd_map = {k:v for k,v in pd.read_csv('clean_stat_missing_CD.csv')[['SegmentID','District']].values}

In [236]:
combined_quarters = combined_quarters.drop(['FullName_Q1_2016', 'LAPD_Grid_Q1_2016','CD_Q1_2016',
                                            'GlobalID_Q1_2016','GlobalID_Q4_2016','Miles_Q1_2016',
                                            'Miles_Q4_2017','FullName_Q4_2016','LAPD_Grid_Q4_2016',
                                           'OBJECTID_1_Q1_2016', 'OBJECTID_1_Q4_2016',
                                            'OBJECTID_1_Q4_2017'],axis=1)

combined_quarters = combined_quarters.rename(columns={'Miles_Q4_2016':'Miles',
                                                      'LAPD_Grid_Q4_2017':'LAPD_Grid',
                                                     'CD_Q4_2016':'CD',
                                                     'FullName_Q4_2017':'FullName'})

combined_quarters = combined_quarters[['FullName','SegmentID','CD','CS_RoundSc_Q1_2016','CS_RoundSc_Q4_2016',
                   'CS_RoundSc_Q4_2017','Miles','SanDist','LAPD_Grid']]


#updating missing CDs
combined_quarters['Updated CD']=combined_quarters['CD']
combined_quarters.loc[combined_quarters['CD']==0,'Updated CD'] = combined_quarters.loc[combined_quarters['CD']==0,'SegmentID'].map(missing_cd_map)


In [237]:

#earlier version to verify true mileage differences
#combined_quarters[['Miles_Q1_2016','Miles_Q4_2016','Miles_Q4_2017']] = combined_quarters[['Miles_Q1_2016','Miles_Q4_2016','Miles_Q4_2017']].round(decimals=5)

street_mapping = {1:'Clean',2:'Somewhat Clean',3:'Not Clean',0:'Not Evaluated'}

combined_quarters['Cat_Q1_2016'],combined_quarters['Cat_Q4_2016'],combined_quarters['Cat_Q4_2017']=\
combined_quarters['CS_RoundSc_Q1_2016'].map(street_mapping),\
combined_quarters['CS_RoundSc_Q4_2016'].map(street_mapping),\
combined_quarters['CS_RoundSc_Q4_2017'].map(street_mapping)

In [365]:
q1_2016_ = combined_quarters.pivot_table('Miles',columns='Cat_Q1_2016',index='CD',aggfunc='sum')[['Clean', 'Somewhat Clean', 'Not Clean', 'Not Evaluated']]
q4_2016_ = combined_quarters.pivot_table('Miles',columns='Cat_Q4_2016',index='CD',aggfunc='sum')[['Clean','Somewhat Clean','Not Clean']]
q4_2017_ = combined_quarters.pivot_table('Miles',columns='Cat_Q4_2017',index='CD',aggfunc='sum')[['Clean','Somewhat Clean','Not Clean']]

q1_2016 = combined_quarters.pivot_table('Miles',columns='Cat_Q1_2016',index='Updated CD',aggfunc='sum')[['Clean', 'Somewhat Clean', 'Not Clean', 'Not Evaluated']]
q4_2016 = combined_quarters.pivot_table('Miles',columns='Cat_Q4_2016',index='Updated CD',aggfunc='sum')[['Clean','Somewhat Clean','Not Clean']]
q4_2017 = combined_quarters.pivot_table('Miles',columns='Cat_Q4_2017',index='Updated CD',aggfunc='sum')[['Clean','Somewhat Clean','Not Clean']]

In [366]:
q1_2016_SouthLA = q1_2016.loc[[8,9,10]]
q1_2016_SouthLA_ = q1_2016_.loc[[8,9,10]]

q4_2016_SouthLA = q4_2016.loc[[8,9,10]]
q4_2016_SouthLA_ = q4_2016_.loc[[8,9,10]]

q4_2017_SouthLA = q4_2017.loc[[8,9,10]]
q4_2017_SouthLA_ = q4_2017_.loc[[8,9,10]]

In [387]:
## ALL LA

all_recoded = reduce(lambda x,y: pd.merge(x,y,left_index=True,right_index=True),
                            [q1_2016,q4_2016,q4_2017]).T

all_no_0 = reduce(lambda x,y: pd.merge(x,y,left_index=True,right_index=True),
                            [q1_2016_,q4_2016_,q4_2017_]).T

#need to remove the 0 districts!
all_no_0 = all_no_0.drop(0,axis=1)

#create new index

updated_index = [x.replace('_y','_Q4_2016') for x in [x.replace('_x','_Q1_2016') for x in all_recoded.index]]
updated_index[3] = 'Not_Evaluated_Q1_2016'
updated_index[-3:]=[x+'_Q4_2017' for x in updated_index[-3:]]


## SOUTH LA

#recoded districts
south_LA_recoded = reduce(lambda x,y: pd.merge(x,y,left_index=True,right_index=True),
                            [q1_2016_SouthLA,q4_2016_SouthLA,q4_2017_SouthLA]).T

#original districts, subtracting "0" districts
south_LA_no_0 = reduce(lambda x,y: pd.merge(x,y,left_index=True,right_index=True),
                            [q1_2016_SouthLA_,q4_2016_SouthLA_,q4_2017_SouthLA_]).T


all_recoded.index,all_no_0.index, south_LA_no_0.index, south_LA_recoded.index = [updated_index]*4



In [302]:
def percent_changes(df,starting,ending):
    raw = (df.sum(axis=1).loc[ending]-df.sum(axis=1).loc[starting])/(df.sum(axis=1).loc[starting])
    return(format(raw*100,'.1f')+'% change')

In [378]:
percent_changes(south_LA_no_0,ending='Clean_Q4_2017',starting='Clean_Q1_2016')

'110.7% change'

In [379]:
percent_changes(south_LA_recoded,ending='Clean_Q4_2017',starting='Clean_Q1_2016')

'119.9% change'

In [388]:
percent_changes(all_no_0,ending='Clean_Q4_2017',starting='Clean_Q1_2016')

'54.4% change'

In [389]:
percent_changes(all_recoded,ending='Clean_Q4_2017',starting='Clean_Q1_2016')

'69.6% change'

In [216]:
os.chdir(outputs)
combined_quarters.to_csv('cleanstat_Q1Q42016_Q42017.csv')